# Study force-controlled single facet

TODO
 - Section 1 - costruct a displacement controlled model and show the load displacement curve for
   sliding
 - Section 2 - with the identified 

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model

In [5]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

In [6]:
plt.rcParams['text.usetex']=True
#plt.rc('text.latex', preamble=r'\usepackage[bitstream-charter]{mathdesign}')
plt.rcParams['font.size'] = 7

# Times, Palatino, New Century Schoolbook, Bookman, Computer Modern Roman
plt.rc('font',**{'family':'serif','serif':['Computer Modern Roman']}) 

import matplotlib
matplotlib.use('pgf')
plt.rcParams["pgf.texsystem"] = "xelatex"
plt.rcParams["pgf.rcfonts"] = False

preamble = r'''\usepackage[utf8]{inputenc} %unicode support
\usepackage[czech]{babel}
\usepackage[T1]{fontenc}
\DeclareMathAlphabet{\pazocal}{OMS}{zplm}{m}{n}
\usepackage{calrsfs}
\usepackage{amsmath}
\usepackage{bm}
\usepackage[bitstream-charter]{mathdesign}
%\usepackage{xcolor}
'''
plt.rc('text.latex', preamble=preamble)
plt.rcParams["pgf.preamble"] = preamble

In [9]:
compression_range = np.array(np.linspace(0,-80/100,3))

m_list = []

for compression in [0]: # compression_range:

    n_e = 1
    L_x = 1.0
    
    xd_lower = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_upper = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_lower.mesh.I[-1, :],
        J=xd_upper.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
    E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_lower, m1),
            (xd_upper, m2),
            (xd12, bond_m),
        ],
    )
    
    tf = TFCyclicNonsymmetricConstant(number_of_cycles=2, unloading_ratio=0.2)
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    upper_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    upper_control_dofs = upper_control_slice.dofs
    upper_control = [BCDof(var='u', dof=dof, value=0.4 ) 
                     for dof in upper_control_dofs ]
    
    F = compression / len(upper_control_dofs)

    upper_compression_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value=0)
    upper_compression_dofs = upper_compression_slice.dofs
    upper_compression = [BCDof(var='f', dof=dof, value=compression ) 
                     for dof in upper_control_dofs ]
    lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = upper_control + upper_compression + [lower_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.005
    s.reset()
    s.run()
    m_list.append(m)

In [10]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2, ax3,  ax4)) =  plt.subplots(1,4, figsize=(16/2.54, 6/2.54)) #, constrained_layout=False)
fig.canvas.header_visible=False
fig.tight_layout()

max_F = []

for i in range(len(m_list)):
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, outer_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }

    
    ax1.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0], label=str(compression_range[i] * 100))
    ax1.set_xlabel('s_x')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()
    
    ax2.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('s_x')
    ax2.set_ylabel('$\sigma_N$')
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    
    max_F.append(max(abs( -EpsSig_t['tau_pi_x'][:,0,0])))
    
fname= 'ElsevierFont.pdf'
plt.savefig(fname, bbox_inches='tight', pad_inches=0.01/2.54, dpi=300)

# get the link to the PDF
from IPython.display import FileLink
FileLink(fname)

/home/rch/PycharmProjects/bmcs_matmod/notebooks/slide/paper_2021/ElsevierFont.pdf

In [17]:
compression_range = np.array(np.linspace(0,-80/100,3))

m_list = []

for i in range(len(max_F)):

    n_e = 1
    L_x = 1.0
    
    xd_inner = XDomainFEGrid(coord_min=(0, -1),
                              coord_max=(1, 0),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    xd_outer = XDomainFEGrid(coord_min=(0, 0),
                              coord_max=(1, 1),
                              shape=(1, 1),
                              fets=FETS2D4Q())
    
    m1 = MATS2DElastic(E=40000, nu=0.2)
    m2 = MATS2DElastic(E=40000, nu=0.2)

    xd12 = XDomainFEInterface(
        I=xd_inner.mesh.I[-1, :],
        J=xd_outer.mesh.I[0, :],
        fets=FETS1D52ULRH()
    )
    
    material_params = dict(
    E_T=100, gamma_T=200, K_T=0, S_T=0.05, c_T=3, bartau=3, 
    E_N=100, S_N=0.05, c_N = 3, m = 0.15, f_t=5, f_c=80, f_c0 = 40, eta=0.2)
    
    bond_m = Slide34(**material_params)
    
    m = TStepBC(
        domains=[
            (xd_inner, m1),
            (xd_outer, m2),
            (xd12, bond_m),
        ],
    )
    
    tf = TFCyclicNonsymmetricConstant(number_of_cycles=2, unloading_ratio=0.05)
    
    load_dim = 0 # 
    fixed_dim = 1 # 0 - x, 1 - y
    
    
    
    outer_control_slice = BCSlice(slice=xd_outer.mesh[:, :, :, :],
                              var='u', dims=[load_dim], value=0)
    outer_control_dofs = outer_control_slice.dofs

    F_max = 0.75 * max_F[i] / len(outer_control_dofs)
    outer_control = [BCDof(var='f', dof=dof, value=F_max, time_function=tf ) 
                         for dof in outer_control_dofs ]
    
    F = compression_range[i] / len(outer_control_dofs)

    outer_ring = BCSlice(slice=xd_outer.mesh[:, :, :, :],
             var='u', dims=[fixed_dim], value= compression_range[i])
    inner_fixed = BCSlice(slice=xd_inner.mesh[:, :, :, :],
             var='u', dims=[0,1], value=0)
    bc1 = outer_control + [outer_ring, inner_fixed]

    m.bc=bc1
    m.hist.vis_record = {
        'strain': Vis3DTensorField(var='eps_ab'),
    }
    s = m.sim
    s.tloop.verbose = False # set false to avoid the prints during each increment
    s.tloop.k_max = 50
    s.tline.step = 0.001
    s.reset()
    s.run()
    m_list.append(m)

RuntimeError: Factor is exactly singular

In [ ]:
import matplotlib.pylab as plt
fig, ((ax1,  ax2, ax3,  ax4)) =  plt.subplots(1,4, figsize=(16/2.54, 6/2.54)) #, constrained_layout=False)
fig.canvas.header_visible=False
fig.tight_layout()

max_F = []

for i in range(len(m_list)):
    m = m_list[i]
    F_to = m.hist.F_t
    U_to = m.hist.U_t
    outer_control_dofs = [bc.dof for bc in outer_control]
    F_l_t = np.sum(F_to[:, outer_control_dofs], axis=-1)
    U_r_t = np.average(U_to[:, outer_control_dofs], axis=-1)
    
    states_t = [states_t[2] for states_t in m.hist.state_vars]
    var_names = states_t[0].keys()
    EpsSig_t = {
        var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
        for var_name in var_names 
    }

    
    ax1.plot(U_r_t, -EpsSig_t['tau_pi_x'][:,0,0], label=str(compression_range[i] * 100))
    ax1.set_xlabel('s_x')
    ax1.set_ylabel('$\sigma_T$')
    ax1.legend()
    
    ax2.plot(U_r_t, -EpsSig_t['sig_pi'][:,0,0])
    ax2.set_xlabel('s_x')
    ax2.set_ylabel('$\sigma_N$')
    
    ax3.plot(U_r_t, -EpsSig_t['omega_T'][:,0,0])
    ax3.set_xlabel('s_x')
    ax3.set_ylabel('$\omega_T$')
    
    
    ax4.plot(U_r_t, -EpsSig_t['omega_N'][:,0,0])
    ax4.set_xlabel('s_x')
    ax4.set_ylabel('$\omega_N$')
    
    max_F.append(max(abs( -EpsSig_t['tau_pi_x'][:,0,0])))
    
fname= 'ElsevierFont.pdf'
plt.savefig(fname, bbox_inches='tight', pad_inches=0.01/2.54, dpi=300)

# get the link to the PDF
from IPython.display import FileLink
FileLink(fname)